<a href="https://colab.research.google.com/github/Santiago-Castellanos/Estudiantes-EAFIT/blob/Renta-Variable---EAFIT/Modelo_3_factores_FF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Instalar paquetes



In [ ]:
!pip install statsmodels
!pip install getFamaFrenchFactors

import pandas as pd
import yfinance as yf
import statsmodels.api as sm
import getFamaFrenchFactors as gff

  Preparing metadata (setup.py) ... done
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1257 sha256=d3def6e3223bd91cce6a192d65f6034057ea6a71ca1c1a981e361b5b5a9588c5
  Stored in directory: /root/.cache/pip/wheels/25/42/45/b773edc52acb16cd2db4cf1a0b47117e2f69bb4eb300ed0e70
Successfully built bs4


## Descargar datos

### Descargar datos acción y calcular retornos

In [ ]:
ticker = 'msft'
start = '2016-8-31'
end = '2023-07-24'

stock_data = yf.download(ticker, start, end)

[*********************100%***********************]  1 of 1 completed


In [ ]:
stock_returns = stock_data['Adj Close'].resample('M').last().pct_change().dropna()
stock_returns.name = "Month_Rtn"

### Descargar información factores

Información descargada desde:
http://mba.tuck.dartmouth.edu/pages/faculty/ken.french/data_library.html

In [ ]:
ff3_monthly = gff.famaFrench3Factor(frequency='m')
ff3_monthly.rename(columns={"date_ff_factors": 'Date'}, inplace=True)
ff3_monthly.set_index('Date', inplace=True)

## Fusionar matriz retornos y facrores

In [ ]:
ff_data = ff3_monthly.merge(stock_returns,on='Date')

In [ ]:
X = ff_data[['Mkt-RF', 'SMB', 'HML']]
y = ff_data['Month_Rtn'] - ff_data['RF']
X = sm.add_constant(X)

## Calcular modelo de regresión lineal

Recordar que al realizar una regresion lineal estamos encontrando los factores (betas) de la siguiente forma:
Y = Xβ+ ϵ
Donde Y es la variable a explicar (retornos de la accion) y X es una matriz compuesta por los factores que me van a explicar el comportamiento de Y.  

In [ ]:
ff_model = sm.OLS(y, X).fit()
print(ff_model.summary())
intercept, b1, b2, b3 = ff_model.params

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.716
Model:                            OLS   Adj. R-squared:                  0.705
Method:                 Least Squares   F-statistic:                     64.76
Date:                Mon, 24 Jul 2023   Prob (F-statistic):           5.28e-21
Time:                        21:53:41   Log-Likelihood:                 167.26
No. Observations:                  81   AIC:                            -326.5
Df Residuals:                      77   BIC:                            -316.9
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0128      0.004      3.577      0.0

## Modelo

In [ ]:
rf = ff_data['RF'].mean()
market_premium = ff3_monthly['Mkt-RF'].mean()
size_premium = ff3_monthly['SMB'].mean()
value_premium = ff3_monthly['HML'].mean()

expected_monthly_return = rf + b1 * market_premium + b2 * size_premium + b3 * value_premium
expected_yearly_return = expected_monthly_return * 12
print("Retorno anual esperado: " + str(expected_yearly_return))

Retorno anual esperado: 0.059528208726503586
